# Cell density and composition

In this notebook you will analyze and validate the circuit anatomy in terms of cell density and composition.

### Cell density

In this section, you will calculate the cell density.

The cell density is defined as the number of cells per volume and is usually expressed as $10^3 / \textrm{mm}^3$.

---

Import some python packages

In [ ]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import nrrd
import matplotlib.pyplot as plt

# package to work with the circuit
from bluepysnap import Circuit
from bluepysnap.bbp import Cell

Reading and preparing the data.

In [ ]:
circuit_path = '/home/data-bbp/20191017/circuit_config.json'
atlas_directory = '/home/data-bbp/20191017/atlas'
circuit = Circuit(circuit_path)
cells = circuit.nodes["hippocampus_neurons"]
conn = circuit.edges["hippocampus_neurons__hippocampus_neurons__chemical"]

In [ ]:
br_data, br_metadata = nrrd.read(Path(atlas_directory, "brain_regions.nrrd"))

# hierarchy contains the region hierarchy: 01 -> [mc0->[mc0;SO, mc0;SP, ...], mc1-> [mc1;SO, mc1;SP, ...], ...]
hierarchy = json.load(open(Path(atlas_directory, "hierarchy.json")))

In [ ]:
O1_columns = hierarchy["children"]
O1_column_names = [column["acronym"] for column in O1_columns]

Initialize table to store the results.

In [ ]:
regions = ['CA1', 'SLM', 'SR', 'SP', 'SO'] # list of regions we are going to analyze
df = pd.DataFrame(index=regions, columns=O1_column_names) # the analysis will be done in the 7 columns separately

Pay attention to the column numbering. This will be critical in other exercises where we focus on central column to avoid boundary effects.

![mosaic_numbering](./images/mosaic_numbering.png)

In [ ]:
df

Since you will apply the same calculation for different cases, better to create a helper function.

In [ ]:
def region_volume(brain_region, brain_region_metadata, labels):
    '''Helper function that takes the region name and gives it total volume
    including the scaling factor to convert from 1/um^3 to 10^3/mm^3'''
    scale = 1000000 # convert factor from 1/um^3 to 10^3/mm^3
    count = np.count_nonzero(np.in1d(brain_region, labels))
    spacing = brain_region_metadata["space directions"].diagonal()
    voxel_volume = abs(np.product(spacing))
    return count*voxel_volume/scale

This is where you calculate the cell density for several subset of cells and in each column.

In [ ]:
for column in O1_columns:
    column_name = column["acronym"]
    
    # retrieve the acronyms and the labels for all brain regions in the corresponding column
    acronyms_labels = [(region['acronym'], region['id']) for region in column["children"]]
    # Get the labels from the brain region atlas corresponding to the current column 
    column_labels = [acronym_label[1] for acronym_label in acronyms_labels]
    
    # cell gids in the current column
    region_str_regex = '{};.*'.format(column_name)
    column_gids = cells.ids({Cell.REGION: {'$regex': region_str_regex}}) 
    
    # compute density 
    column_density = len(column_gids)/region_volume(br_data, br_metadata, column_labels)
    
    # fill the data frame
    df.loc["CA1"][column_name] = column_density
    
    for acronym, label in acronyms_labels:
        # get the gids for the current region
        region_gids = cells.ids({Cell.REGION: acronym})
        # compute the density of the region
        region_density = len(region_gids)/ region_volume(br_data, br_metadata, label)
        # fill the dataframe
        df.loc[acronym.split(";")[-1]][column_name] = region_density

In [ ]:
df

Calculate mean and standard deviation across the columns.

In [ ]:
means = df.mean(axis=1)
stds = df.std(axis=1)
df['mean'] = means
df['std'] = stds
df.head()

### Cell composition

In this section you count how many cells you have for each morphological types or m-types and in each 'column'.

You have already prepared the data in the first part, you only need to query the different m-types in the circuit.

In [ ]:
mtypes = cells.property_values(Cell.MTYPE)

Initialize table to store the results.

In [ ]:
composition = pd.DataFrame(index=mtypes, columns=O1_column_names)
composition.head()

Compute the number of cells for each m-types and in each column.

In [ ]:
# calculate the number of cells
for column in O1_columns:
    column_acronym = column["acronym"]
    region_str_regex = '{};.*'.format(column_acronym)
    for mtype in mtypes:
        composition[column_acronym][mtype] = cells.count({Cell.MTYPE: mtype, Cell.REGION: {'$regex': region_str_regex}})

In [ ]:
composition

Calculate mean and standard deviation across the columns.

In [ ]:
means = composition.mean(axis=1)
stds = composition.std(axis=1)
composition['mean'] = means
composition['std'] = stds
composition

### EI ratio

In this section, you calculate the EI ratio, i.e. the ratio between the number of Excitatory and Inhibitory cells. EI ratio is often express as percentages of cells.

The EI ratio gives you an idea of the balance between excitation and inhibition, and that may have a profound effect on the network activity.

---

You have already prepared the data in the first part, you only need to initialize the table to store the results.

In [ ]:
rows = ['EXC', 'INH', 'EXC%']

ratio = pd.DataFrame(index=rows, columns=O1_column_names)

Calculate the number of Excitatory and Inhibitory cells in each column.

In [ ]:
for column in O1_columns:
    column_acronym = column['acronym']
    region_str_regex = '{};.*'.format(column_acronym)
    for target in ['EXC', 'INH']:
        ratio[column_acronym][target] = cells.count({Cell.SYNAPSE_CLASS: target, Cell.REGION: {'$regex': region_str_regex}})

Convert the excitatory cell counts to percentage and fill the corresponding row in the table

In [ ]:
# calculate percentage of exhitatory cells (EI ratio)
ratio.loc['EXC%'] = ratio.loc['EXC']*100/(ratio.loc['EXC']+ratio.loc['INH'])
ratio

Calculate mean and standard deviation across the columns

In [ ]:
means = ratio.mean(axis=1)
stds = ratio.std(axis=1)
ratio['mean'] = means
ratio['std'] = stds
ratio.head()

### Cells positions across layers

Another important aspect of the circuit anatomy is the location of the cells.

In this section, you will show soma locations in different layers.

---

Query the soma location in the space.

In [ ]:
df = cells.positions({Cell.REGION: {'$regex': 'mc2;.*'}})
df.head()

Plot the soma locations to visualize their relationship with layers.

In [ ]:
# extract values for plotting
x = df['x'].values
y = df['y'].values

In [ ]:
# plot the results
fig, ax = plt.subplots()
ax.scatter(x,y)
layer_heights = (0, 170, 230, 510, 660)
heights = (80, 190, 365, 580)
layers = ('SO', 'SP', 'SR', 'SLM')
ax.hlines(layer_heights, x.min(), x.max(), linestyle='--', colors='r')
for layer, height in zip(layers, heights):
    ax.text(x.min(), height, layer, fontsize=12, color='r')
fig.show()

### Exercise #1
The cell densities calculated above should be compared with literature. Extract data from literature (Aika et al 1994) and compare with the model.

![Aika_etal_1994_Table3](./images/Aika_etal_1994_Table3.png)

__Hints__
The table provides more info than you need. For this exercise, focus only on the total number of cells in the entire CA1 and in the different layers. In addition to those 4 datapoints, consider also the PC density in SP.
Furthermore, note that Aika et al considered SLM and SR together (SRLM).

Store answer in two lists, one with data extracted from literature and one with data extracted from the model, each with the order density in CA1 - SRLM - SP - PC density in SP - SO. Store the answers in variables _ans_1a_ and _ans_1b_.

### Exercise #2
Location for CCK and SOM cells. Answer will be a comma separated string from SLM to SO without space. For example, location of PV cells is 'SP'. Store answer in two lists, respectively for CCK and SOM.Store the answers in variables _ans_2a_ and _ans_2b_ 

### Exercise #3
Calculate density for PV, CCK, and SOM cells. Divide the number to the layer volume where they are present. Store answers in a list _ans3_ respecting the order PV, CCK, and SOM.

In [ ]:
# Work here 

In [ ]:
# This is to generate the answers to paste in the submission box below.
# Run this and copy-paste the output into the box below
print(json.dumps(dict([("ans_1a", ans_1a),
                       ("ans_1b", ans_1b),
                       ("ans_2a", ans_2a),
                       ("ans_2b", ans_2b),
                       ("ans_3", ans_3)])))

!pip -q install -i https://bbpteam.epfl.ch/repository/devpi/simple/ single-cell-mooc-client==0.0.5 
import single_cell_mooc_client as sc_mc
s = sc_mc.Submission(hideToken=True)

In [ ]:
# Show submission widget
s.show_submission(ROLLBACK, TOKEN)